In [ ]:
import geopandas as gpd
import pandas as pd
from h3 import h3
from sqlalchemy import create_engine
from shapely.geometry import Polygon, Point
import numpy as np
import psycopg2

In [ ]:

def h3_from_row(row,res,x,y):
    '''
    Esta funcion toma una fila, un nivel de resolucion de h3
    y los nombres que contienen las coordenadas xy
    y devuelve un id de indice h3
    '''
    return h3.geo_to_h3(row[y],row[x],res = res)

def h3_indexing(df,res_list,lat='LATITUDE',lon='LONGITUDE'):
    """
    Esta funcion toma una tabla con dos pares de coordenadas para origen y destino
    Un nivel de resolucion h3
    y devuelve la tabla con los ids de h3
    """
    
    if len(res_list) == 1:
        res_list.append(res_list[0])
        
    for res in range(res_list[0],res_list[1] + 1):
        df['h3_res_' + str(res)] = df.apply(h3_from_row, axis=1, args=[res,lon,lat])

    return df

def vertices_cada_Xmetros(geom,metros):
    n_puntos = int((geom.length/metros)+1)
    percentiles = np.linspace(0,geom.length,n_puntos)
    return [geom.interpolate(percentil,normalized=False) for percentil in percentiles]

def convertir_recorridos_buses_paradas(fila,metros=400):
    paradas = vertices_cada_Xmetros(fila.geometry,metros = metros)
    paradas = gpd.GeoSeries(paradas).map(lambda g: Point(g.coords[0][0:2]))
    crs = 'EPSG:3857'
    gdf = gpd.GeoDataFrame(np.repeat(fila.LINEA,len(paradas)),crs=crs,geometry=paradas)
    gdf.columns=['LINEA','geometry']
    return gdf

In [ ]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [ ]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [ ]:
# traer modos
sql = """
select distinct t."LINEA"
from public.trx t
where t."MEDIO" = 'SUB'
"""
lineas_subte_trx = pd.read_sql(sql, conn)

In [ ]:
lineas_subte_trx

In [ ]:
subte = gpd.read_file('../carto/insumos/subterraneo-estaciones')
subte = subte.reindex(columns = ['LINEA','geometry'])

premetro = pd.read_csv('../carto/insumos/estaciones-premetro.csv')
premetro = gpd.GeoDataFrame(
    premetro, geometry=gpd.points_from_xy(premetro['long'], premetro.lat),crs='EPSG:4326')
premetro = premetro.reindex(columns = ['linea','geometry'])
premetro.columns = ['LINEA','geometry']

subte = pd.concat([subte,premetro])
subte['MEDIO'] = 'SUB'
subte['LINEA'] = 'LINEA '+subte['LINEA']

In [ ]:
lineas_subte_trx.isin(subte.LINEA.unique())

In [ ]:
# traer modos
sql = """
select distinct t."LINEA"
from public.trx t
where t."MEDIO" = 'TRE'
"""
lineas_ffcc_trx = pd.read_sql(sql, conn)
lineas_ffcc_trx

In [ ]:
ffcc = gpd.read_file('../carto/insumos/rmba-ferrocarril-estaciones/')
ffcc = ffcc.loc[ffcc.Tipo=='Estación',['Línea','geometry']]
ffcc.columns = ['LINEA','geometry']
ffcc['MEDIO'] = 'TRE'

In [ ]:
ffcc.LINEA.unique()

In [ ]:
ffcc_equivalencias = {'Mitre':'SOFSE - Mitre',
                      'Sarmiento':'SOFSE- SARMIENTO',
                      'Roca':'SOFSE - Roca',
                      'Belgrano Sur':'SOFSE - Belgrano Sur',
                      'San Martín':'SOFSE - San Martin',
                      'Belgrano Norte':'FERROVIAS S.A.',
                      'Urquiza':'METROVIAS S.A. (URQUIZA)'}

In [ ]:
ffcc.LINEA = ffcc.LINEA.replace(ffcc_equivalencias)

In [ ]:
lineas_ffcc_trx.isin(ffcc.LINEA.unique())

## Buses

In [ ]:
# nacionales
bus_nac = gpd.read_file('../carto/insumos/lineas-nacionales/')
bus_nac = bus_nac.reindex(columns = ['LINEA','geometry'])
bus_nac = bus_nac.to_crs('EPSG:3857')

paradas_bus_nac = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_nac.iterrows()])
paradas_bus_nac.crs = 'EPSG:3857'
paradas_bus_nac = paradas_bus_nac.to_crs('EPSG:4326')
paradas_bus_nac['MEDIO'] = 'COL'

# provinciales
bus_prov = gpd.read_file('../carto/insumos/lineas-provinciales/')
bus_prov = bus_prov.reindex(columns = ['LINEA','geometry'])
bus_prov = bus_prov.to_crs('EPSG:3857')

paradas_bus_prov = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_prov.iterrows()])
paradas_bus_prov.crs = 'EPSG:3857'
paradas_bus_prov = paradas_bus_prov.to_crs('EPSG:4326')
paradas_bus_prov['MEDIO'] = 'COL'
paradas_bus_prov.head()

# municipales
bus_muni = gpd.read_file('../carto/insumos/lineas-municipales/')
bus_muni = bus_muni.reindex(columns = ['LINEA','geometry'])
bus_muni = bus_muni.to_crs('EPSG:3857')

paradas_bus_muni = pd.concat([convertir_recorridos_buses_paradas(fila) for i,fila in bus_muni.iterrows()])
paradas_bus_muni.crs = 'EPSG:3857'
paradas_bus_muni = paradas_bus_muni.to_crs('EPSG:4326')
paradas_bus_muni['MEDIO'] = 'COL'

In [ ]:
bus = pd.concat([paradas_bus_muni,paradas_bus_prov,paradas_bus_nac])


In [ ]:
bus['LINEA'] = 'LINEA '+bus['LINEA'].map(str)

In [ ]:
# traer modos
sql = """
select distinct t."LINEA"
from public.trx t
where t."MEDIO" = 'COL'
"""
lineas_bus_trx = pd.read_sql(sql, conn)
lineas_bus_trx

In [ ]:
#faltan 1/3
lineas_faltantes = lineas_bus_trx[~lineas_bus_trx.isin(bus.LINEA.unique()).values]
len(lineas_faltantes)

In [ ]:
#separar las municipales y ver si se puede usar alguna forma de detectar a que linea pertenecen

In [ ]:
no_500 = lineas_faltantes.LINEA.map(lambda s: s[:7] != 'LINEA 5')
si_500 = ~no_500

In [ ]:
lineas_faltantes[si_500].head()

In [ ]:
lineas_faltantes[no_500].head()

In [ ]:
paradas = pd.concat([bus,ffcc,subte])
paradas.head()

In [ ]:
paradas.shape

In [ ]:
paradas.to_file('../carto/paradas.geojson',driver='GeoJSON')

In [ ]:
paradas['LATITUDE'] = paradas.geometry.y
paradas['LONGITUDE'] = paradas.geometry.x
paradas.drop('geometry',axis=1,inplace=True)

In [ ]:
conn.close()

In [ ]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'
    .format(DB_USERNAME, DB_PASSWORD, DB_HOST,
            DB_PORT, DB_NAME))

In [ ]:
%time h3_paradas = h3_indexing(paradas.copy(),res_list = [5,12])

In [ ]:
h3_paradas.head()

In [ ]:
h3_paradas.to_csv('../data/h3_paradas.csv',index=False)

In [ ]:
h3_paradas.to_sql('paradas', engine, schema=DB_SCHEMA)

In [ ]:
# plotear el mapa de hexagrillas de paradas
lista_indices_global = h3_paradas.h3_res_5.unique()
geo_df = gpd.GeoDataFrame(lista_indices_global,
                          geometry = [Polygon(h3.h3_to_geo_boundary(h3_address=h, geo_json=True)
                                                                   ) for h in lista_indices_global],
                          crs = 'EPSG:4326')
geo_df.columns=['h3_index','geometry']
geo_df.to_file('../carto/carto_paradas_hex_res_5.geojson',driver='GeoJSON')
geo_df.head()